In [1]:
import pandas as pd
import os

In [2]:
path_folder = "/Users/matteolemesre/Desktop/Data LOSC/csv/csv24_25/"
path_start  = os.path.join(path_folder, "clean/data_players.csv")
path_end_aggregated_data = os.path.join(path_folder, "centiles/data_players_aggregated.csv")
path_end_adjusted_data   = os.path.join(path_folder, "centiles/data_players_adjusted.csv")
path_end_centiles        = os.path.join(path_folder, "centiles/data_players_centiles.csv")

In [3]:
data = pd.read_csv(path_start, index_col=0)

In [4]:
def aggregated_data_players(data):
    df_grouped = data.groupby("Player").agg({
        "Team": lambda x: " / ".join(sorted(set(x))),
        "League": lambda x: " / ".join(sorted(set(x))),
        "Nationality": "first",
        "Position": "first",
        "Age": "first",
        "Minutes": "sum",
        
        "Goals": "sum",
        "Assists": "sum",
        "Shots Total": "sum",
        "Shots on Target": "sum",
        "Expected Goals (xG)": "sum",
        "Non-Penalty Expected Goals (npxG)": "sum",
        "Penalty Kicks Made": "sum",
        "Penalty Kicks Attempted": "sum",
        "Shot-Creating Actions (SCA)": "sum",
        "Goal-Creating Actions (GCA)": "sum",
        "Key Passes": "sum",
        "Passes into Final Third": "sum",
        "Passes into Penalty Area": "sum",
        "Crosses into Penalty Area": "sum",
        "Crosses": "sum",

        "Expected Assists (xA)": "sum",
        "Expected Assisted Goals (xAG)": "sum",

        "Passes Completed": "sum",
        "Passes Attempted": "sum",
        "Progressive Passes": "sum",
        "Through Balls": "sum",
        "Switches": "sum",
        "Passes Blocked": "sum",
        "Passes Offside": "sum",
        "Live-ball Passes": "sum",
        "Dead-ball Passes": "sum",
        "Passes from Free Kicks": "sum",

        "Passes Completed (Short)": "sum",
        "Passes Attempted (Short)": "sum",
        "Passes Completed (Medium)": "sum",
        "Passes Attempted (Medium)": "sum",
        "Passes Completed (Long)": "sum",
        "Passes Attempted (Long)": "sum",

        "Passing Distance (Total)": "sum",
        "Progressive Passing Distance (Total)": "sum",

        "Carries": "sum",
        "Progressive Carries": "sum",
        "Carrying Distance (Total)": "sum",
        "Progressive Carrying Distance (Total)": "sum",
        "Carries into Final Third": "sum",
        "Carries into Penalty Area": "sum",

        "Take-Ons Attempted": "sum",
        "Successful Take-Ons": "sum",
        "Times Tackled During Take-On": "sum",

        "Tackles": "sum",
        "Tackles Won": "sum",
        "Tackles in Defensive Third": "sum",
        "Tackles in Middle Third": "sum",
        "Tackles in Attacking Third": "sum",

        "Dribblers Tackled": "sum",
        "Dribbles Challenged": "sum",
        "Challenges Lost": "sum",

        "Interceptions": "sum",
        "Blocks": "sum",
        "Shots Blocked": "sum",
        "Passes Blocked": "sum",
        "Clearances": "sum",
        "Errors Leading to Shot": "sum",

        "Ball Recoveries": "sum",
        "Miscontrols": "sum",
        "Dispossessed": "sum",
        "Ball Losses": "sum",
        "Passes Received": "sum",
        "Progressive Passes Received": "sum",

        "Touches": "sum",
        "Touches in Defensive Penalty Area": "sum",
        "Touches in Defensive Third": "sum",
        "Touches in Middle Third": "sum",
        "Touches in Attacking Third": "sum",
        "Touches in Attacking Penalty Area": "sum",
        "Live-Ball Touches": "sum",

        "Yellow Cards": "sum",
        "Red Cards": "sum",
        "Fouls Committed": "sum",
        "Fouls Drawn": "sum",
        "Offsides": "sum",
        "Penalty Kicks Conceded": "sum",
        "Penalty Kicks Won": "sum",
        "Own Goals": "sum",

        "Aerials Won": "sum",
        "Aerials Lost": "sum",
        "Total Aerials": "sum"
    }).reset_index()

    df_grouped.insert(df_grouped.columns.get_loc("Minutes"), "Matches", data.groupby("Player")["Game Week"].count().values)

    df_grouped["Passing Distance"] = df_grouped["Passing Distance (Total)"] / df_grouped["Passes Attempted"]
    df_grouped["Carrying Distance"] = df_grouped["Carrying Distance (Total)"] / df_grouped["Carries"]
    df_grouped["Progressive Pass Distance %"] = df_grouped["Progressive Passing Distance (Total)"] / df_grouped["Passing Distance (Total)"]
    df_grouped["Progressive Carry Distance %"] = df_grouped["Progressive Carrying Distance (Total)"] / df_grouped["Carrying Distance (Total)"]
    
    columns_to_drop = [
        "Passing Distance (Total)", "Carrying Distance (Total)",
        "Progressive Passing Distance (Total)", "Progressive Carrying Distance (Total)"
    ]
    df_grouped = df_grouped.drop(columns=columns_to_drop)

    for col in df_grouped.select_dtypes(include=['float64', 'int64']).columns:
        df_grouped[col] = df_grouped[col].round(2)

    return df_grouped

In [5]:
def adjusted_data_players(data):
    stats_per_90 = data.copy()
   
    stats_columns = [
        "Goals", "Assists", "Shots Total", "Shots on Target",
        "Expected Goals (xG)", "Non-Penalty Expected Goals (npxG)",
        "Penalty Kicks Made", "Penalty Kicks Attempted",
        "Shot-Creating Actions (SCA)", "Goal-Creating Actions (GCA)",
        "Key Passes", "Passes into Final Third", "Passes into Penalty Area",
        "Crosses into Penalty Area", "Crosses", "Expected Assists (xA)",
        "Expected Assisted Goals (xAG)", "Passes Completed", "Passes Attempted",
        "Progressive Passes", "Through Balls", "Switches", "Passes Blocked",
        "Passes Offside", "Live-ball Passes", "Dead-ball Passes",
        "Passes from Free Kicks", "Passes Completed (Short)", "Passes Attempted (Short)",
        "Passes Completed (Medium)", "Passes Attempted (Medium)", "Passes Completed (Long)",
        "Passes Attempted (Long)", "Carries", "Progressive Carries",
        "Carries into Final Third", "Carries into Penalty Area",
        "Take-Ons Attempted", "Successful Take-Ons", "Times Tackled During Take-On",
        "Tackles", "Tackles Won", "Tackles in Defensive Third", "Tackles in Middle Third",
        "Tackles in Attacking Third", "Dribblers Tackled", "Dribbles Challenged",
        "Challenges Lost", "Interceptions", "Blocks", "Shots Blocked",
        "Clearances", "Errors Leading to Shot", "Ball Recoveries", "Miscontrols",
        "Dispossessed", "Ball Losses", "Passes Received", "Progressive Passes Received",
        "Touches", "Touches in Defensive Penalty Area", "Touches in Defensive Third",
        "Touches in Middle Third", "Touches in Attacking Third",
        "Touches in Attacking Penalty Area", "Live-Ball Touches",
        "Yellow Cards", "Red Cards", "Fouls Committed", "Fouls Drawn", "Offsides",
        "Penalty Kicks Conceded", "Penalty Kicks Won", "Own Goals",
        "Aerials Won", "Aerials Lost", "Total Aerials"
    ]
    
    for column in stats_columns:
        if column in stats_per_90.columns:
            stats_per_90[column] = stats_per_90[column] * 90 / stats_per_90["Minutes"]

    for col in stats_per_90.select_dtypes(include=['float64', 'int64']).columns:
        stats_per_90[col] = stats_per_90[col].round(2)
    
    return stats_per_90

In [6]:
def centiles_data_players(data):
    centiles = data.copy()

    stats_columns = [
        "Goals", "Assists", "Shots Total", "Shots on Target",
        "Expected Goals (xG)", "Non-Penalty Expected Goals (npxG)",
        "Penalty Kicks Made", "Penalty Kicks Attempted",
        "Shot-Creating Actions (SCA)", "Goal-Creating Actions (GCA)",
        "Key Passes", "Passes into Final Third", "Passes into Penalty Area",
        "Crosses into Penalty Area", "Crosses", "Expected Assists (xA)",
        "Expected Assisted Goals (xAG)", "Passes Completed", "Passes Attempted",
        "Progressive Passes", "Through Balls", "Switches", "Passes Blocked",
        "Passes Offside", "Live-ball Passes", "Dead-ball Passes",
        "Passes from Free Kicks", "Passes Completed (Short)", "Passes Attempted (Short)",
        "Passes Completed (Medium)", "Passes Attempted (Medium)", "Passes Completed (Long)",
        "Passes Attempted (Long)", "Carries", "Progressive Carries",
        "Carries into Final Third", "Carries into Penalty Area",
        "Take-Ons Attempted", "Successful Take-Ons", "Times Tackled During Take-On",
        "Tackles", "Tackles Won", "Tackles in Defensive Third", "Tackles in Middle Third",
        "Tackles in Attacking Third", "Dribblers Tackled", "Dribbles Challenged",
        "Challenges Lost", "Interceptions", "Blocks", "Shots Blocked",
        "Clearances", "Errors Leading to Shot", "Ball Recoveries", "Miscontrols",
        "Dispossessed", "Ball Losses", "Passes Received", "Progressive Passes Received",
        "Touches", "Touches in Defensive Penalty Area", "Touches in Defensive Third",
        "Touches in Middle Third", "Touches in Attacking Third",
        "Touches in Attacking Penalty Area", "Live-Ball Touches",
        "Yellow Cards", "Red Cards", "Fouls Committed", "Fouls Drawn", "Offsides",
        "Penalty Kicks Conceded", "Penalty Kicks Won", "Own Goals",
        "Aerials Won", "Aerials Lost", "Total Aerials", "Passing Distance", "Carrying Distance", "Progressive Pass Distance %", "Progressive Carry Distance %"
    ]

    stats_columns_neg = [
        "Challenges Lost", "Errors Leading to Shot", "Miscontrols",
        "Dispossessed", "Ball Losses", "Yellow Cards", "Red Cards",
        "Fouls Committed", "Offsides", "Penalty Kicks Conceded", "Own Goals"
    ]

    for column in stats_columns_neg:
        if column in centiles.columns:
            centiles[column] = -centiles[column]

    for column in stats_columns:
        if column in centiles.columns:
            centiles[column] = centiles[column].rank(pct=True) * 100

    for col in centiles.select_dtypes(include=['float64', 'int64']).columns:
        centiles[col] = centiles[col].round(2)

    return centiles

In [7]:
df_aggregated = aggregated_data_players(data)
df_adjusted = adjusted_data_players(df_aggregated)
df_centiles = centiles_data_players(df_adjusted)
df_aggregated.to_csv(path_end_aggregated_data, index=False)
df_adjusted.to_csv(path_end_adjusted_data, index=False)
df_centiles.to_csv(path_end_centiles, index=False)